In [40]:
import io
import requests
import pandas as pd

from tqdm.auto import tqdm

In [83]:
def get_data_table(aid):
    url = f"https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={aid}"
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text), delimiter=',')
        return df
    else:
        return None
    
def preproc_df(df, assay_name):
    df = df[df['PUBCHEM_EXT_DATASOURCE_SMILES'].notna()]
    df = df.loc[:,df.columns[df.isna().mean(0) != 1.0]]
    df = df.drop(['PUBCHEM_RESULT_TAG', 'PUBCHEM_SID', 'PUBCHEM_CID'], axis = 1)
    df['text'] = df.apply(lambda row: str(row[1:]), axis=1)
    df['smiles'] = df['PUBCHEM_EXT_DATASOURCE_SMILES']
    df['query'] = assay_name
    df['answer'] = df['text']
    return df[['smiles', 'query', 'answer']]

In [37]:
data = pd.read_csv('data/PubChem_bioassay_text_Liver microsomes.csv')

C:\Users\dust\AppData\Local\Temp\ipykernel_16636\1042594141.py:1: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/PubChem_bioassay_text_Liver microsomes.csv')


In [90]:
dfs = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    try:
        aid = row['aid']
        assay_name = row['aidname']
        df = get_data_table(aid)
        if df is None: continue
        df = preproc_df(df, assay_name)
        dfs.append(df)
    except Exception as e:
        print(aid, e)
        continue

  0%|          | 0/34085 [00:00<?, ?it/s]

In [89]:
df

,smiles,query,answer
2,COC1=CC(=CC(=C1OC)OC)C(=O)C2=CC=CC(=C2)C3=CC4=...,Metabolic stability in rat liver microsomes as...,PUBCHEM_ACTIVITY_OUTCOME Unspecified\nStand...


In [18]:
url = 'https://pubchem.ncbi.nlm.nih.gov/bioassay/157#section=Data-Table&embed=true'

In [20]:
response = requests.get(url)

In [23]:
if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to retrieve the web page.")

In [ ]:
https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid=4447

In [30]:
import requests
import pandas as pd


# Define the URL of the CSV file
url = "https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid=4447"

# Send an HTTP GET request to download the CSV file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Read the CSV data into a pandas DataFrame
    df = pd.read_csv(io.StringIO(response.text), delimiter=',')

    # Now, df contains the data from the CSV file, and you can work with it as needed
    # For example, you can print the first few rows of the DataFrame
    print(df.head())
else:
    print("Failed to download the CSV file")


                         PUBCHEM_RESULT_TAG  PUBCHEM_SID  PUBCHEM_CID  \
0                               RESULT_TYPE          NaN          NaN   
1                              RESULT_DESCR          NaN          NaN   
2                               RESULT_UNIT          NaN          NaN   
3            RESULT_IS_ACTIVE_CONCENTRATION          NaN          NaN   
4  RESULT_IS_ACTIVE_CONCENTRATION_QUALIFIER          NaN          NaN   

  PUBCHEM_EXT_DATASOURCE_SMILES PUBCHEM_ACTIVITY_OUTCOME  \
0                           NaN                      NaN   
1                           NaN                      NaN   
2                           NaN                      NaN   
3                           NaN                      NaN   
4                           NaN                      NaN   

   PUBCHEM_ACTIVITY_SCORE  PUBCHEM_ACTIVITY_URL  PUBCHEM_ASSAYDATA_COMMENT  \
0                     NaN                   NaN                        NaN   
1                     NaN                   

In [43]:
df = df[df['PUBCHEM_EXT_DATASOURCE_SMILES'].notna()]

In [53]:
df = df.loc[:,df.columns[df.isna().mean(0) != 1.0]]

In [58]:
df = df.drop(['PUBCHEM_RESULT_TAG', 'PUBCHEM_SID', 'PUBCHEM_CID'], axis = 1)

In [65]:
if len(df)>=2:
    df.loc[:,df.nunique()!=1]

In [70]:
columns = df.columns[1:]

In [75]:
query = '(' + ', '.join(columns) + ') of this molecule is:'

In [80]:
df = df.drop('text',axis=1)

In [81]:
df.apply(lambda row: str(row[1:]), axis=1)

5     PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
6     PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
7     PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
8     PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
9     PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
10    PUBCHEM_ACTIVITY_OUTCOME    Unspecified\nPubCh...
11    PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
12    PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem St...
13    PUBCHEM_ACTIVITY_OUTCOME    Unspecified\nPubCh...
dtype: object

In [79]:
df['text'].values[0]

'(PUBCHEM_ACTIVITY_OUTCOME, PubChem Standard Value, Standard Type, Standard Relation, Standard Value, Standard Units) of this molecule is:PUBCHEM_ACTIVITY_OUTCOME    Active\nPubChem Standard Value       0.024\nStandard Type                   Ki\nStandard Relation                =\nStandard Value                  24\nStandard Units                  nM\nName: 5, dtype: object'